In [1]:
import gadgets
import datasets
import transformers
import pandas as pd
import os

[2023-10-24 12:36:02,064] [INFO] [real_accelerator.py:133:get_accelerator] Setting ds_accelerator to cuda (auto detect)


In [2]:
ds = datasets.load_dataset("gsm8k", "main")

In [3]:
assert set(ds.keys()) == {"train", "test"}

In [4]:
calc = gadgets.gadget.Calculator()

df = {}
for split in ds.keys():
    df[split] = ds[split].to_pandas()
    examples = df[split].apply(gadgets.gsm8k.parse, axis=1)
    df[split].drop(columns=["answer"], inplace=True)
    df[split]["chain"] = examples.apply(lambda x: gadgets.markup.to_model_markup(example=x)).apply(str)
    df[split]["result"] = examples.apply(lambda x: x.result)
    df[split]["result_float"] = df[split]["result"].apply(calc._float_eval)

/home/xkadlci2/gadgets/gadgets/markup.py:14: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  return bs4.BeautifulSoup(step, features="html.parser")
/home/xkadlci2/gadgets/gadgets/markup.py:14: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  return bs4.BeautifulSoup(step, features="html.parser")


In [5]:
ds_orig_split = datasets.DatasetDict({
    "train": datasets.Dataset.from_pandas(df["train"]),
    "test": datasets.Dataset.from_pandas(df["test"]),
})

# ds_orig_split.push_to_hub("MU-NLPC/Calc-gsm8k", config_name="original-splits")

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/8 [00:00<?, ?ba/s]

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

In [6]:
print("ORIGINAL SPLIT:")
print(ds_orig_split)
seed = 0

ds_train_new_split, ds_valid_new_split = ds_orig_split["train"].train_test_split(seed=seed, test_size=200).values()
ds_new_split = datasets.DatasetDict({
    "train": ds_train_new_split,
    "validation": ds_valid_new_split,
    "test": ds_orig_split["test"],
})
print("NEW SPLIT:")
print(ds_new_split)

ORIGINAL SPLIT:
DatasetDict({
    train: Dataset({
        features: ['question', 'chain', 'result', 'result_float'],
        num_rows: 7473
    })
    test: Dataset({
        features: ['question', 'chain', 'result', 'result_float'],
        num_rows: 1319
    })
})
NEW SPLIT:
DatasetDict({
    train: Dataset({
        features: ['question', 'chain', 'result', 'result_float'],
        num_rows: 7273
    })
    validation: Dataset({
        features: ['question', 'chain', 'result', 'result_float'],
        num_rows: 200
    })
    test: Dataset({
        features: ['question', 'chain', 'result', 'result_float'],
        num_rows: 1319
    })
})


In [7]:
# ds_new_split.push_to_hub("MU-NLPC/Calc-gsm8k")

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/8 [00:00<?, ?ba/s]

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

In [8]:
datasets.load_dataset("MU-NLPC/Calc-gsm8k")

DatasetDict({
    test: Dataset({
        features: ['question', 'chain', 'result', 'result_float'],
        num_rows: 1319
    })
    train: Dataset({
        features: ['question', 'chain', 'result', 'result_float'],
        num_rows: 7273
    })
    validation: Dataset({
        features: ['question', 'chain', 'result', 'result_float'],
        num_rows: 200
    })
})

In [9]:
datasets.load_dataset("MU-NLPC/Calc-gsm8k", "original-splits")

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating test split:   0%|          | 0/1319 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/7473 [00:00<?, ? examples/s]

DatasetDict({
    test: Dataset({
        features: ['question', 'chain', 'result', 'result_float'],
        num_rows: 1319
    })
    train: Dataset({
        features: ['question', 'chain', 'result', 'result_float'],
        num_rows: 7473
    })
})